In [1]:
!date

Fri Apr 28 05:58:15 EDT 2023


In [2]:
pwd

'/mmfs1/data/aglinska/BC-actionpred-seg'

In [3]:
import PIL
from PIL import Image,ImageFilter
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
from scipy.ndimage.filters import gaussian_filter
import os
import shutil
import random
#import cv2
from skimage.transform import resize
from datetime import datetime
now = datetime.now

In [4]:
def load_video(fn):
    import skvideo
    import skvideo.io
    videodata = skvideo.io.vread(fn)
    return videodata

In [5]:
def safe_mkdir(fn):
    import os
    if not os.path.exists(fn):
        os.mkdir(fn)

In [6]:
dataset_dir = './Data/dataset-v3-10/'
categories = [cat for cat in os.listdir(dataset_dir) if not cat.startswith('.')]
categories.sort()
n = len(categories)
print(n)
print(categories[0:3])
print(categories[-3::])

45
['CPR', 'badminton_overswing', 'basketball_pass']
['weightlifting_stand', 'whipping', 'yoga_updog']


In [7]:
outdir = './Data/pytorch-data/action_data_rnd'
#fn_temp = '{this_cat}_{vid_num:03d}.mp4' # original videos
#fn_temp = '{this_cat}_{vid_num:03d}_item_0_fwd_seg.mp4' # Segmented videos
#fn_temp = '{this_cat}_{vid_num:03d}_item_0_inp_seg.mp4' # Background
fn_temp = '{this_cat}_{vid_num:03d}_item_0_bg_rnd.mp4' # RND inpainted

safe_mkdir(outdir)

safe_mkdir(os.path.join(outdir,'train'))
safe_mkdir(os.path.join(outdir,'val'))
safe_mkdir(os.path.join(outdir,'test'))

[safe_mkdir(os.path.join(outdir,'train',category)) for category in categories];
[safe_mkdir(os.path.join(outdir,'val',category)) for category in categories];
[safe_mkdir(os.path.join(outdir,'test',category)) for category in categories];

#cat_val = 8
#cat_test = 9

In [8]:
for c in tqdm( range( n ) ):
    this_cat = categories[c]
    files = os.listdir(os.path.join(dataset_dir,this_cat))
    files.sort()
    vid_nums = np.unique([int(file.replace(this_cat+'_','')[0:3]) for file in files])
    vid_nums = vid_nums[0:10]
    
    for vid_num in vid_nums:
        video_data = load_video(os.path.join(dataset_dir,this_cat,fn_temp.format(this_cat=this_cat,vid_num=vid_num))) # load video 360*640
        nframes = video_data.shape[0]
        video_data = resize(np.pad(video_data,((0,0),(140,140),(0,0),(0,0))),(nframes,224,224,3)) # resize to 244*244 (0-1 range)
        video_data = (video_data*255).astype(np.uint8) # make into 0-255 uint

        if vid_num==vid_nums[8]:
            outdir_subfolder = os.path.join(outdir,'val',this_cat)
        elif vid_num==vid_nums[9]:
            outdir_subfolder = os.path.join(outdir,'test',this_cat)
        else:
            outdir_subfolder = os.path.join(outdir,'train',this_cat)    

        # Save all frames
        [Image.fromarray(video_data[f,:,:,:]).save(os.path.join(outdir_subfolder,f'{vid_num:03d}_{f:03d}.jpg')) for f in range(nframes)];

100%|██████████| 45/45 [1:26:00<00:00, 114.69s/it]
